In [53]:
from mainnet_launch.data_fetching.get_state_by_block import (
    get_state_by_one_block,
    build_blocks_to_use,
    get_raw_state_by_blocks,
    safe_normalize_6_with_bool_success,
    safe_normalize_with_bool_success,
)

import os
import pandas as pd
from tqdm import tqdm
import concurrent.futures as cf

from mainnet_launch.database.postgres_operations import (
    get_full_table_as_df_with_block,
    get_full_table_as_df,
    get_full_table_as_df_with_tx_hash,
)
from mainnet_launch.database.schema.full import (
    DestinationStates,
    Destinations,
    AutopoolDestinations,
    RebalanceEvents,
    RebalancePlans,
    Blocks,
)
import pandas as pd

from multicall import Call
import numpy as np
import time
import datetime
from mainnet_launch.data_fetching.defi_llama.fetch_timestamp import fetch_blocks_by_unix_timestamps_defillama
from mainnet_launch.data_fetching.internal.s3_helper import fetch_rebalance_plan_json_no_s3_client
import plotly.express as px
import plotly.io as pio


from mainnet_launch.constants import (
    AUTO_USD,
    ETH_CHAIN,
    AUTO_ETH,
    AutopoolConstants,
    AUTO_DOLA,
    BASE_USD,
    BASE_CHAIN,
    ALL_AUTOPOOLS,
    BASE_ETH
)

from mainnet_launch.data_fetching.internal.s3_helper import  fetch_rebalance_plan_json_no_s3_client

pio.templates.default = None


autopool = AUTO_ETH

destinations = get_full_table_as_df(Destinations, where_clause=Destinations.chain_id == autopool.chain.chain_id)
autopool_destinations = get_full_table_as_df(
    AutopoolDestinations, where_clause=AutopoolDestinations.autopool_vault_address == autopool.autopool_eth_addr
)
# # 2 min to fetch
# destination_states = get_full_table_as_df_with_block(
#     DestinationStates,
#     where_clause=DestinationStates.destination_vault_address.in_(
#         destinations["destination_vault_address"].tolist()
#     ),
# )
plans = get_full_table_as_df(
    RebalancePlans, where_clause=RebalancePlans.autopool_vault_address == autopool.autopool_eth_addr
).sort_values('datetime_generated')
events = get_full_table_as_df_with_tx_hash(
    RebalanceEvents, where_clause=RebalanceEvents.autopool_vault_address == autopool.autopool_eth_addr
).sort_index()
mainnet_blocks = get_full_table_as_df(Blocks, where_clause=Blocks.chain_id == autopool.chain.chain_id).sort_values(
    "block"
)

In [54]:
events

,tx_hash,autopool_vault_address,chain_id,rebalance_file_path,destination_out,destination_in,quantity_out,quantity_in,safe_value_out,safe_value_in,spot_value_in,spot_value_out,spot_value_in_solver_change,block
datetime,,,,,,,,,,,,,,
2024-09-17 19:42:23+00:00,0xf4b0efb2e5d69ef7b7d000b4860a9d86b76013d9a042...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1726602027_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xE382BBd32C4E202185762eA433278f4ED9E6151E,100.000000,99.961014,100.000000,99.963008,99.966261,100.000000,0.000000,20772483
2024-09-18 04:02:23+00:00,0xfdb95550d0720ff1f6ca2ff7331d1eb3194fe85ee8ab...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1726632022_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xE382BBd32C4E202185762eA433278f4ED9E6151E,314.500371,314.380624,314.500371,314.372549,314.387618,314.500371,0.000000,20774967
2024-09-18 16:22:35+00:00,0xaa013ce865573532459a55487fb809c3e93edeed05af...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1726676459_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,146.034135,144.676019,146.034135,145.966127,145.967367,146.034135,0.021905,20778643
2024-09-19 02:57:47+00:00,0x1e5ddbb3a2e56692c1c261b05b482298c1cd6bd6bcf3...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1726714560_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xd96E943098B2AE81155e98D7DC8BeaB34C539f01,134.705433,133.437132,134.705433,134.644712,134.648167,134.705433,0.020206,20781802
2024-09-19 13:26:23+00:00,0xfb4645f276bf56d5ff8fef87301516ccb8ed0841a5db...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1726752261_0x0A2b94F6871c1D7A32...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,0xE382BBd32C4E202185762eA433278f4ED9E6151E,335.831303,335.615197,335.831303,335.660996,335.663713,335.831303,-0.042111,20784928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-12 12:31:11+00:00,0x3980b42c8756999dcd7401c330c685027b5291261999...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1768220996_0x0A2b94F6871c1D7A32...,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,47.263918,39.475138,47.366768,47.283073,47.292483,47.326576,0.011828,24218647
2026-01-12 22:31:11+00:00,0xa00be293f95ab78c3b64c2ef8cd2411a3e4d0a2aa309...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1768256996_0x0A2b94F6871c1D7A32...,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,33.191028,27.731277,33.205006,33.216349,33.222772,33.227399,0.008311,24221631
2026-01-13 14:30:59+00:00,0x078c6ea3b5e542eb59f58a9056f69646c331620e671c...,0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56,1,rebalance_plan_1768314596_0x0A2b94F6871c1D7A32...,0x43D99d04985ef2231f7d9b5D9111D2189d9FD971,0x2C7120dCCF1c14A37A26A4955475d45d34a3d7E7,24.893271,20.796678,24.936594,24.909304,24.917280,24.937640,0.006235,24226402


In [55]:
latest_auto_eth_plan = events.iloc[-1]['rebalance_file_path']
latest_executed_plan = fetch_rebalance_plan_json_no_s3_client(latest_auto_eth_plan, autopool)
executed_sod = pd.json_normalize(latest_executed_plan['sod']['destStates'])

latest_plan_generated = plans.iloc[-1]['file_name']
latest_generated_plan = fetch_rebalance_plan_json_no_s3_client(latest_plan_generated, autopool)
generated_timestamp = datetime.datetime.fromtimestamp(latest_generated_plan['timestamp'], tz=datetime.timezone.utc)
print(f"Generated timestamp: {generated_timestamp}")
generated_sod = pd.json_normalize(latest_generated_plan['sod']['destStates'])   

Generated timestamp: 2026-02-04 02:29:56+00:00


In [56]:
generated_sod[['name', 'safePrice', 'spotPrice']]

,name,safePrice,spotPrice
0,Tokemak-Wrapped Ether-Balancer osETH-waWETH,1.023956,1.023980
1,Tokemak-Wrapped Ether-Curve.fi Factory Pool: E...,1.054835,1.054858
2,Tokemak-Wrapped Ether-Instadapp ETH v2,1.199944,1.200461
3,Tokemak-Wrapped Ether-osETH/rETH,1.063982,1.063999
4,Tokemak-Wrapped Ether-Balancer Surge Fluid wst...,1.021170,1.021273
5,Tokemak-Wrapped Ether-weETH/WETH-ng,1.065457,1.065405
6,Tokemak-Wrapped Ether-Wrapped liquid staked Et...,1.224846,0.112854


In [57]:
latest_generated_plan

{'timestamp': 1770172196,
 'sodOnly': True,
 'chainId': '1',
 'solverAddress': '0x952D7a7eB2e0804d37d9244BE8e47341356d2f5D',
 'poolAddress': '',
 'destinationOut': '',
 'tokenOut': '',
 'amountOut': '',
 'amountOutETH': '',
 'destinationIn': '',
 'tokenIn': '',
 'minAmountIn': '',
 'minAmountInETH': '',
 'steps': [],
 'sod': {'currentTimestamp': 1770172196,
  'destStates': [{'snapshotTimestamp': 0,
    'name': 'Tokemak-Wrapped Ether-Balancer osETH-waWETH',
    'address': '0x87E25fFe5C3A2720Cd43F5Eb8ec41Ac0CE699D07',
    'poolType': 'balV3',
    'pool': '0x57c23c58B1D8C3292c15BEcF07c62C5c52457A42',
    'underlying': '0x57c23c58B1D8C3292c15BEcF07c62C5c52457A42',
    'underlyingTokens': ['0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
     '0xf1C9acDc66974dFB6dEcB12aA385b9cD01190E38'],
    'underlyingTokenAmounts': [553867067968944017239, 949357803538907686339],
    'underlyingReserves': [553867850988227216514, 948112703593146102530],
    'totalAprIn': 0.051691330770640066,
    'totalAprOut